<a href="https://colab.research.google.com/github/kimjeongsoo20190147/dacon/blob/main/dacon_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# Import Libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from joblib import Parallel, delayed
import multiprocessing
import tqdm
import torch.multiprocessing as mp

In [44]:
# Load Data


train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [45]:
# Data Preprocessing - 수정된 부분 (형태소 분석 제거)
# Combine title and keywords
train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

In [46]:
# Prepare train and validation data
X = train_df['제목_키워드']
y = train_df['분류']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [47]:
# Feature Extraction using CountVectorizer and TfidfVectorizer
# You can choose either CountVectorizer or TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

# Combine TF-IDF features
X_train_combined = X_train_vec.toarray()
X_val_combined = X_val_vec.toarray()

In [48]:
# Model Training
model = MultinomialNB()
model.fit(X_train_combined, y_train)

MultinomialNB()

In [1]:
# Validation
y_val_pred = model.predict(X_val_combined)
val_f1 = f1_score(y_val, y_val_pred, average='macro')
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation F1 Score: {val_f1:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

NameError: name 'model' is not defined

In [50]:
# Epoch-wise Training and Validation - 수정된 부분
CFG = {'epoch': 3}
for epoch in range(CFG['epoch']):
    print(f"Epoch {epoch + 1}/{CFG['epoch']}")
    model.fit(X_train_combined, y_train)

    # Validation after each epoch
    y_val_pred = model.predict(X_val_combined)
    val_f1 = f1_score(y_val, y_val_pred, average='macro')
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Validation F1 Score: {val_f1:.2f}")
    print(f"Validation Accuracy: {val_accuracy:.2f}")

Epoch 1/3
Validation F1 Score: 0.32
Validation Accuracy: 0.70
Epoch 2/3
Validation F1 Score: 0.32
Validation Accuracy: 0.70
Epoch 3/3
Validation F1 Score: 0.32
Validation Accuracy: 0.70


In [22]:
# Test Data Preparation and Prediction
X_test = test_df['제목_키워드']
X_test_vec = vectorizer.transform(X_test)

X_test_combined = X_test_vec.toarray()

y_test_pred = model.predict(X_test_combined)

# Save Submission
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission['분류'] = y_test_pred
sample_submission.to_csv("submission_konlp_tfidf_kobert.csv", index=False, encoding='utf-8-sig')